In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
from fastai import *
from fastai.text import *

# **1.First I need to train a language model**

In [0]:
path = untar_data(URLs.IMDB_SAMPLE)
data_lm = TextLMDataBunch.from_csv(path, 'texts.csv')

In [0]:
learn_lm = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [0]:
learn_lm.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.325931,3.912266,0.282733,00:22


In [0]:
learn_lm.save_encoder('enc')

# **2. This is what a think is how to get the sentence representation**

In [0]:
def masked_concat_pool(outputs, mask):
    "Pool MultiBatchEncoder outputs into one vector [last_hidden, max_pool, avg_pool]."
    output = outputs[-1]
    avg_pool = output.masked_fill(mask[:, :, None], 0).mean(dim=1)
    avg_pool *= output.size(1) / (output.size(1)-mask.type(avg_pool.dtype).sum(dim=1))[:,None]
    max_pool = output.masked_fill(mask[:,:,None], -float('inf')).max(dim=1)[0]
    x = torch.cat([output[:,-1], max_pool, avg_pool], 1)
    return x

In [0]:
xb,yb = learn_lm.data.one_item('i did not like this movie because it was really bad.')
sentence = torch.cuda.LongTensor(xb.tolist()[0]).unsqueeze(0)
mbe = MultiBatchEncoder(70,1400,learn_lm.model[0])
raw_outputs, outputs, masks = mbe.forward(sentence)
sentence_rep = masked_concat_pool(outputs, masks)

In [0]:
sentence_rep.shape

torch.Size([1, 1200])

# **3. Lets see what accuracy fastai gets**

In [0]:
data_clas = TextClasDataBunch.from_csv(path,'texts.csv',vocab=data_lm.vocab)

In [0]:
learn_clas = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn_clas.load_encoder('enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (799 items)
x: TextList
xxbos xxmaj this is a romantic comedy with the emphasis on comedy for a change . xxmaj as usual the lovers -- xxmaj sally xxmaj field as almost - over - the - hill soap opera queen , xxmaj celeste xxmaj xxunk ; and xxmaj kevin xxmaj kline as marginally employed and marginally talented actor , xxmaj jeffrey xxmaj anderson -- are working at cross purposes , seemingly unaware that they are xxunk in love , etc . xxmaj xxunk a little to xxmaj bette xxmaj davis 's xxmaj margo xxmaj xxunk in xxmaj all xxmaj about xxmaj eve ( 1950 ) and a whole lot to the slapstick theatrical tradition , xxmaj sally xxmaj field goes over the top towards hilarity as she xxunk her way to love and happiness . xxmaj kevin xxmaj kline , one of the more underrated leading men of recent years , is also very good and very winning as he manages to be handsome , vulnerable , xxunk and lovable all at the same time . 
 
  xxmaj the xxunk center a

In [0]:
learn_clas.fit_one_cycle(5)

epoch,train_loss,valid_loss,accuracy,time
0,0.644827,0.673254,0.661692,00:14
1,0.592541,0.620803,0.671642,00:15
2,0.537566,0.517414,0.761194,00:14
3,0.503054,0.482044,0.791045,00:14
4,0.476742,0.468615,0.791045,00:14


# **4. Now lets see what we get using the representations we've got**

In [0]:
mbe = MultiBatchEncoder(70,1400,learn_lm.model[0])

In [0]:
X_train = np.zeros(1200)
for i in range(799):
  xb,yb = learn_lm.data.one_item(data_clas.train_ds.x[i])
  sentence = torch.cuda.LongTensor(xb.tolist()[0]).unsqueeze(0)
  raw_outputs, outputs, masks = mbe.forward(sentence)
  sentence_rep = masked_concat_pool(outputs, masks)
  X_train = np.vstack((X_train,sentence_rep))

In [0]:
X_valid = np.zeros(1200)
for i in range(201):
  xb,yb = learn_lm.data.one_item(data_clas.valid_ds.x[i])
  sentence = torch.cuda.LongTensor(xb.tolist()[0]).unsqueeze(0)
  raw_outputs, outputs, masks = mbe.forward(sentence)
  sentence_rep = masked_concat_pool(outputs, masks)
  X_valid = np.vstack((X_valid,sentence_rep))

In [0]:
y_train = np.zeros(799)
for i in range(799):
  y_train[i] = int(data_clas.train_ds.y[i])

In [0]:
y_valid = np.zeros(201)
for i in range(201):
  y_valid[i] = int(data_clas.valid_ds.y[i])

In [0]:
X_train = np.delete(X_train, (0), axis=0)
X_valid = np.delete(X_valid, (0), axis=0)

In [0]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(alpha=1, max_iter=1000,)
clf.fit(X_train,y_train)
clf.score(X_valid,y_valid)

0.746268656716418